In [97]:
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from pyvi import ViTokenizer
import pandas as pd
import numpy as np

2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [ ]:
%ls -la

total 225
drwx------ 3 root root  4096 Apr 14 03:53 data/
-rw------- 1 root root 63670 Apr 26 11:00 data_processing.ipynb
drwx------ 2 root root  4096 Apr 14 04:17 model/
-rw------- 1 root root  4482 Apr 26 04:26 scrape_news.ipynb
-rw------- 1 root root 21991 Apr 26 08:35 search_generate_text.ipynb
drwx------ 2 root root  4096 Apr 26 03:59 src/
drwx------ 2 root root  4096 Apr 14 09:19 temp/
-rw------- 1 root root 44172 Apr 26 08:37 text_generator.ipynb
-rw------- 1 root root 78007 Apr 26 11:13 word_embedding.ipynb


In [ ]:
news = pd.read_csv('./data/vnexpress/csv/vnexpress_processed.csv').drop(columns='Unnamed: 0', axis=1)
news.head()

,article_id,content,topic,sub-topic,title,description
0,0,"Đoàn đại biểu Ban Chấp hành Trung ương Đảng, C...",thoi-su,chinh-tri,"Lãnh đạo Đảng, Nhà nước vào Lăng viếng Chủ tịc...","Sáng 26/4, lãnh đạo Đảng, Nhà nước đặt v..."
1,1,Tại hội nghị phát triển nguồn nhân lực phục vụ...,thoi-su,chinh-tri,Bộ trưởng Nguyễn Mạnh Hùng: Lương cao sẽ thu h...,Hà NộiTheo Bộ trưởng Thông tin và Truyền thông...
2,2,"Tại kỳ họp 40 (ngày 23-24/4), Ủy ban Kiểm tra ...",thoi-su,chinh-tri,"Nguyên Chủ tịch tỉnh An Giang, Sóc Trăng bị kỷ...",Nguyên Chủ tịch tỉnh An Giang Vương Bình Thạnh...
3,3,"Ngày 24/4, Ủy ban Kiểm tra (UBKT) Trung ương p...",thoi-su,chinh-tri,Hàng chục cán bộ tỉnh Vĩnh Phúc bị kỷ luật,"Nguyên chủ tịch UBND, HĐND, trưởng Ban Tổ chức..."
4,4,Tại phiên họp lần thứ 8 của Ủy ban Quốc gia về...,thoi-su,chinh-tri,Thủ tướng yêu cầu Bộ Công an mở rộng tiện ích ...,Thủ tướng Phạm Minh Chính thúc đẩy mở rộng tiệ...


In [ ]:
news['tag'] = news['content'] + news['topic'] + news['sub-topic'] + news['title'] + news['description']
news = news[['tag']]

In [ ]:
news.head()

,tag
0,"Đoàn đại biểu Ban Chấp hành Trung ương Đảng, C..."
1,Tại hội nghị phát triển nguồn nhân lực phục vụ...
2,"Tại kỳ họp 40 (ngày 23-24/4), Ủy ban Kiểm tra ..."
3,"Ngày 24/4, Ủy ban Kiểm tra (UBKT) Trung ương p..."
4,Tại phiên họp lần thứ 8 của Ủy ban Quốc gia về...


In [ ]:
news.isna().sum()

tag    3
dtype: int64

In [ ]:
news.dropna(axis=0, inplace=True)

In [ ]:
news['tag'] = news['tag'].apply(lambda x: x.lower())
news['tag'] = news['tag'].apply(ViTokenizer.tokenize)

In [ ]:
sequences = news['tag'].to_list()
input_gensim = []

for sen in sequences:
  input_gensim.append(sen.split())

model = Word2Vec(input_gensim, vector_size=50, window=5, min_count=0, workers=4, sg=1)
model.wv.save("./model/word.model")

In [120]:
question = 'trong giây_phút đoàn_tụ , gia_đình ôm chặt lấy nhau khóc nức_nở'

def embedding(question, input_gensim):
  question_tokens = ViTokenizer.tokenize(question)
  print(question_tokens)

  # Word embeddings
  question_embeddings = [model.wv[word] for word in question_tokens if word in model.wv]

  # Post embeddings
  post_check = input_gensim[:20]

  post_embeddings = []
  for post in post_check:
      post_embedding = [model.wv[word] for word in post if word in model.wv]
      post_embeddings.append(post_embedding)

  return question_embeddings, post_embeddings

question_embeddings, post_embeddings = embedding(question, input_gensim)

trong giây_phút đoàn_tụ , gia_đình ôm chặt lấy nhau khóc nức_nở


In [104]:
question_embeddings[0]

array([-0.23258011, -0.31862968, -0.14273325,  0.08205283, -0.22706506,
       -0.2875181 ,  0.38882437,  0.47883293, -0.10034607,  0.05926759,
        0.09618699, -0.2154017 , -0.01032853, -0.23546387, -0.35390255,
        0.10639068, -0.26079106, -0.42935124, -0.05760948, -0.07749783,
       -0.05637337,  0.4172929 ,  0.12909356, -0.05620804, -0.12561828,
        0.2931681 , -0.28282848, -0.22504747, -0.07846542, -0.02576129,
        0.02431758,  0.35846436, -0.43066582,  0.11266702,  0.1691933 ,
        0.11795394, -0.0669219 ,  0.20739254, -0.06440234, -0.3326181 ,
        0.09140065, -0.20025733, -0.21238354,  0.01135087,  0.41909248,
        0.2243098 ,  0.24906398, -0.18008678, -0.03126693,  0.19530877],
      dtype=float32)

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

In [121]:
# Calculate sentence embeddings by averaging word embeddings
def mean_sentence_embed(embeddings):
  if len(embeddings) == 0:
      return np.zeros(model.vector_size)
  return np.mean(embeddings, axis=0)

mean_sentence_embedding = mean_sentence_embed(question_embeddings)

def mean_post_embed(post_embeddings):
  mean_post_embedding = []
  for post_embedding in post_embeddings:
      mean_post_embedding.append(mean_sentence_embed(post_embedding))

  return mean_post_embedding

mean_post_embedding = mean_post_embed(post_embeddings)

# Calculate similarity (cosine similarity)
similarity_score = cosine_similarity([mean_sentence_embedding], mean_post_embedding)

# Print the similarity score
print(f"Similarity Score: {similarity_score}")

Similarity Score: [[0.92972857 0.88666034 0.90253335 0.87973213 0.9092615  0.8848928
  0.91922665 0.9087951  0.89452565 0.8794235  0.8927279  0.8729906
  0.92889446 0.9350879  0.9380058  0.9177068  0.8773311  0.92677116
  0.8438409  0.9008677 ]]


In [122]:
# Convert the list of lists into a numpy array
np_similarity_score = np.array(similarity_score)
print('np_question_embedding', np_similarity_score)
# Sort the array in ascending order
sorted_indices = np.argsort(np_similarity_score[0])[::-1]

# Print the sorted array
print("sorted_indices", sorted_indices)

# Get the top 5 indices
top_5_indices = sorted_indices[:5]

# Print the sorted indices and the top 5 indices
print("Sorted Indices:", sorted_indices)
print("Top 5 Indices:", top_5_indices)

np_question_embedding [[0.92972857 0.88666034 0.90253335 0.87973213 0.9092615  0.8848928
  0.91922665 0.9087951  0.89452565 0.8794235  0.8927279  0.8729906
  0.92889446 0.9350879  0.9380058  0.9177068  0.8773311  0.92677116
  0.8438409  0.9008677 ]]
sorted_indices [14 13  0 12 17  6 15  4  7  2 19  8 10  1  5  3  9 16 11 18]
Sorted Indices: [14 13  0 12 17  6 15  4  7  2 19  8 10  1  5  3  9 16 11 18]
Top 5 Indices: [14 13  0 12 17]


In [123]:
for i, indice in enumerate(top_5_indices):
    # Remove duplicate spaces
    cleaned_sentence = ' '.join(sequences[indice][:200].split())
    print("Bài viết thứ " + str(indice) + ":", cleaned_sentence)

Bài viết thứ 14: từ 6h15 , đoàn nghi_lễ gồm lãnh_đạo đảng , nhà_nước , tiêu_binh , đoàn , hội các tỉnh , thành_phố , quận huyện của tỉnh phú thọ bắt_đầu lễ rước từ chân núi nghĩa lĩnh lên đền hùng . từ 6h15 , đoàn ngh
Bài viết thứ 13: lãnh_đạo chính_phủ hoan_nghênh công_nhân vẫn làm_việc miệt_mài trên công_trường dự_án ngay trong ngày nghỉ lễ với tinh_thần " chỉ bàn làm , không bàn_lùi , thi_công 3 ca , 4 kíp , vượt nắng , thắng mư
Bài viết thứ 0: đoàn đại_biểu ban chấp_hành trung_ương đảng , chủ_tịch nước , quốc_hội , chính_phủ , ủy_ban trung_ương mặt_trận tổ_quốc việt nam sau đó dâng hương , tưởng_niệm liệt_sỹ tại đài tưởng_niệm trên đường bắ
Bài viết thứ 12: ngày 19 / 4 , tại kỳ họp thứ 14 , 100 % đại_biểu hđnđ có_mặt đã biểu_quyết bãi_nhiệm chức_vụ chủ_tịch hđnd tỉnh với ông trần đức quận , cựu bí_thư tỉnh_ủy . đồng_thời , ông trần văn hiệp cũng bị bãi_n
Bài viết thứ 17: theo thủ_tướng , thời_gian qua , đảng , nhà_nước đã ban_hành nhiều chủ_trương , chính_sách để hỗ_trợ thương , bệ